In [9]:
import dvc.api
import csv

In [16]:
genres_file = dvc.api.read(
    'data/data_by_genres_o.csv',
    repo='https://github.com/konradszewczyk/Spotify_DataMining'
)

ImportError: cannot import name 'gcd' from 'fractions' (C:\Python39\lib\fractions.py)

In [14]:
print(genres_file)

In [15]:
genres_csv = csv.reader(genres_file)

TypeError: '_OpenContextManager' object is not iterable